In [12]:
import os
os.environ["MLFLOW_DISABLE_GIT_COMMIT_TRACKING"] = "true"
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
import tempfile
import logging
import shutil
from urllib.parse import urlparse

logging.basicConfig(level=logging.INFO)

In [13]:
# import os
os.chdir(r"C:\Users\Mayank Goel\OneDrive\Desktop\Chest Cancer Classification Using ML Flow")

In [14]:
%pwd

'C:\\Users\\Mayank Goel\\OneDrive\\Desktop\\Chest Cancer Classification Using ML Flow'

In [15]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mayank2004201/Chest-Cancer-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mayank2004201"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9919caffab5e1bfcca980761377f7d1097a715eb"

In [16]:
import tensorflow as tf

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [18]:
from src.Chest_Cancer_Classification.constants import *
from src.Chest_Cancer_Classification.utils.common import read_yaml,create_directories,save_json

In [19]:

class ConfigurationManager:
    def __init__( 
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
        path_of_model="artifacts/training/model.h5",
        training_data="artifacts/data_ingestion/Chest-CT-Scan_data",
        mlflow_uri="https://dagshub.com/mayank2004201/Chest-Cancer-Classification.mlflow",
        all_params=self.params,
        params_image_size=self.params.IMAGE_SIZE,
        params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [20]:
import tensorflow as tf
from pathlib import Path
import mlflow
import dagshub
import mlflow.keras
import tempfile
import logging
import shutil
from urllib.parse import urlparse

In [21]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1.0/255, validation_split=0.30)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        try:
            if mlflow.active_run() is not None:
                mlflow.end_run()
            
            with mlflow.start_run():
                # your logging code...
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
        finally:
            # Always clean up at the end
            if mlflow.active_run() is not None:
                mlflow.end_run()

In [22]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation() 
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

INFO:Chest Cancer Classification Logger:YAML file: config\config.yaml loaded successfully.
INFO:Chest Cancer Classification Logger:YAML file: params.yaml loaded successfully.
INFO:Chest Cancer Classification Logger:Created directory at: artifacts


Found 299 images belonging to 4 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5719 - loss: 1.0914


INFO:Chest Cancer Classification Logger:Json file saved at: scores.json
2026/02/02 15:38:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/02 15:38:46 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2026/02/02 15:38:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/02/02 15:39:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


🏃 View run unequaled-calf-88 at: https://dagshub.com/mayank2004201/Chest-Cancer-Classification.mlflow/#/experiments/0/runs/f1fb8ea6778a4ec594096651ac3a9809
🧪 View experiment at: https://dagshub.com/mayank2004201/Chest-Cancer-Classification.mlflow/#/experiments/0
